### 2. An Array of Sequence


##### Types of Sequences
- Build-in sequences:
    - container sequences: can hold items of different types, list, tuple, and collections.deque
    - Flat sequences: hold items of one type, str, bytes, bytearray, memoryview, and array.array
- another way of grouping:
    - mutable sequences: list, bytearray, array.array, collections.deque, memoryview
    - immutable sequences: tuple, str, and bytes


##### list comprehesions and Generator Expressions
- list comprehesion: more readable; you should keep it short
- genexp: saves memory, yields items one by one using the iterator protocol


##### Tuples are not just immutable Lists
- Tuples used as records: the number of items is fixed and their order is vital
- Tuple unpacking
    - swapping the values of variables without using a temporary variable
    - prefixing an argument with a star * when calling a function
    - use * to grab excess items
    - Nested Tuple unpacking: expression should match the nesting structure
- Named Tuples
    - collections.namedtuple
    - use same memory as tuple, but less memory as regular object
    - _fields: a tuple of field names of the class
    - _make(iterable): instantiate a named tuple from an iterable
    - _asdict(): returns OrderedDict build from namedtuple instance
- Tuples as Immutable Lists
    - tuple supports all list methods except adding or removing items
    

##### Slicing
- seq[start:stop:step]
- slice object: slice(a, b, c)
- seq[start:stop:step]  -->  seq.\_\_getitem\_\_(slice(start, stop, step))


##### sorting
- list.sort sorts a list in place, return None (cannot cascade calls)
- sorted create and return a new list
    - accept any interable object including immutable sequences and generators
- managing ordered Sequences with bisect
    - use binanry search algorithm to quickly find and insert items in any sorted sequence
    - searching with **bisect.bisect**, **bisect.bisect_left**
    - inserting with **bisect.insort**


##### Other types of Sequences
- Arrays: more **efficient** if the list only constains numbers
- Memory Views: shared-memory sequence, handle slices of arrays without copying bytes
- Numpy and Scipy: advanced array and matrix operations, scientific computing algorithms
- collections.deque: a thread-safe, double-ended queue designed for fast inserting and removing from both ends
    - discard items from the opposite end when queue is full and append items
- queue.Queue, queue.LifoQueue, queue.PriorityQueue; used for safe communication between threads. 
    - insertion is blocked when queue is full, instead of discarding items
- multiprocessing queue: designed for iterprocess communication
- asycio queue: Queue, LifoQueue, PriorityQueue, JoinableQueue; manage tasks in asynchronous programming
- heapq: provide functions like heappush, heappop

In [6]:
# Example of listcomp and genexp
colors = ['black', 'white']
sizes = ['S', 'M', 'L']
tshirts = [(color, size) for color in colors 
                         for size in sizes]
print(tshirts)

for tshirt in ('%s %s' % (c, s) for c in colors 
                                for s in sizes):
    print(tshirt)

[('black', 'S'), ('black', 'M'), ('black', 'L'), ('white', 'S'), ('white', 'M'), ('white', 'L')]
black S
black M
black L
white S
white M
white L


In [13]:
# Example of Tuple unpacking 1
# swapping the values of variables without using a temporary variable
a = 1
b = 2
print(a, b)
b, a = a, b
print(a, b)

# prefixing an argument with a star when calling a function
# use dummy variable like _ as a placeholder
r = divmod(20, 8)
print(r)
t = (20, 8)
#r = divmod(t)
_, mod = divmod(*t)
print(mod)

1 2
2 1
(2, 4)
4


In [14]:
# Example of Tuple unpacking 1
# use * to grab excess items
a, b, *rest = range(5)
print(a, b, rest)
a, b, *rest = range(2)
print(a, b, rest)
a, *body, c, d = range(5)
print(a, body, c, d)
a, *body, c, d = range(3)
print(a, body, c, d)
*head, b, c, d = range(5)
print(head, b, c, d)

0 1 [2, 3, 4]
0 1 []
0 [1, 2] 3 4
0 [] 1 2
[0, 1] 2 3 4


In [17]:
# Example of namedtuple
from collections import namedtuple

City = namedtuple('City', 'name country population coordinates')
tokyo = City('Tokyo', 'JP', 36.933, (35.689722, 139.691667))
print(tokyo)
print(tokyo.population)
print(tokyo.coordinates)

City(name='Tokyo', country='JP', population=36.933, coordinates=(35.689722, 139.691667))
36.933
(35.689722, 139.691667)


In [23]:
# Example of namedtuple 2
print(City._fields)
LatLong = namedtuple('LatLong', 'lat long')
delhi_data = ('Delhi NCR', 'IN', 21.935, LatLong(28.613889, 77.208889))
delhi = City._make(delhi_data)
#delhi = City(*delhi_data)
print(delhi._asdict())
#for key, value in delhi._asdict().items():
#    print(key + ':',  value)

('name', 'country', 'population', 'coordinates')
{'name': 'Delhi NCR', 'country': 'IN', 'population': 21.935, 'coordinates': LatLong(lat=28.613889, long=77.208889)}


In [28]:
# Example of slice object
invoice = '''
0.....6.................................40........52...55........
1909  Pimoroni PiBrella                     $17.50    3    $52.50
1489  6mm Tactile Switch x20                 $4.95    2     $9.90
1510  Panavise Jr. - PV-201                 $28.00    1    $28.00
1601  PiTFT Mini Kit 320x240                $34.95    1    $34.95
'''

SKU = slice(0, 6)
DESCRIPTION = slice(6, 40)
UNIT_PRICE = slice(40, 52)
QUANTITY = slice(52, 55)
ITEM_TOTAL = slice(55, None)
line_items = invoice.split('\n')[2:]
for item in line_items:
    print(item[DESCRIPTION].strip(), '\t', item[UNIT_PRICE].strip())

Pimoroni PiBrella 	 $17.50
6mm Tactile Switch x20 	 $4.95
Panavise Jr. - PV-201 	 $28.00
PiTFT Mini Kit 320x240 	 $34.95
 	 


In [40]:
# example of bisect.bisect 1
import bisect
import sys

HAYSTACK = [1, 4, 5, 6, 8, 12, 15, 20, 21, 23, 23, 26, 29, 30]
NEEDLES = [0, 1, 2, 5, 8, 10, 22, 23, 29, 30, 31]

ROW_FMT = '{0:2d} @ {1:2d}    {2}{0:<2d}'

def demo(bisect_fn):
    for needle in reversed(NEEDLES):
        position = bisect_fn(HAYSTACK, needle)
        offset = position * '  |'
        print(ROW_FMT.format(needle, position, offset))

bisect_fn = bisect.bisect
print('DEMO', bisect_fn.__name__)
print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
demo(bisect_fn)
print()
bisect_fn = bisect.bisect_left
print('DEMO', bisect_fn.__name__)
print('haystack ->', ' '.join('%2d' % n for n in HAYSTACK))
demo(bisect_fn)

DEMO bisect_right
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |29
23 @ 11      |  |  |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  5      |  |  |  |  |8 
 5 @  3      |  |  |5 
 2 @  1      |2 
 1 @  1      |1 
 0 @  0    0 

DEMO bisect_left
haystack ->  1  4  5  6  8 12 15 20 21 23 23 26 29 30
31 @ 14      |  |  |  |  |  |  |  |  |  |  |  |  |  |31
30 @ 13      |  |  |  |  |  |  |  |  |  |  |  |  |30
29 @ 12      |  |  |  |  |  |  |  |  |  |  |  |29
23 @  9      |  |  |  |  |  |  |  |  |23
22 @  9      |  |  |  |  |  |  |  |  |22
10 @  5      |  |  |  |  |10
 8 @  4      |  |  |  |8 
 5 @  2      |  |5 
 2 @  1      |2 
 1 @  0    1 
 0 @  0    0 


In [42]:
# example of bisect.bisect 2

def grade(score, breakpoints=[60, 70, 80, 90], grades='FDCBA'):
    i = bisect.bisect(breakpoints, score)
    return grades[i]
[grade(score) for score in [33, 99, 77, 70, 89, 90, 100, 61, 60, 59]]

['F', 'A', 'C', 'C', 'B', 'A', 'A', 'D', 'D', 'F']

In [43]:
# example of bisect.insort
import bisect 
import random

SIZE = 7
random.seed(1729)

my_list = []
for i in range(SIZE):
    new_item = random.randrange(SIZE * 2)
    bisect.insort(my_list, new_item)
    print('%2d ->' % new_item, my_list)

10 -> [10]
 0 -> [0, 10]
 6 -> [0, 6, 10]
 8 -> [0, 6, 8, 10]
 7 -> [0, 6, 7, 8, 10]
 2 -> [0, 2, 6, 7, 8, 10]
10 -> [0, 2, 6, 7, 8, 10, 10]


In [49]:
# example of array
from array import array 
from random import random

floats = array('d', (random() for i in range(10**3)))
print(len(floats), floats[:3])

floats2 = array('d')
for i in range(10):
    floats2.append(random())
print(floats2)

floats3 = array(floats2.typecode, sorted(floats2))
print(floats3)

1000 array('d', [0.8014104164650118, 0.5161205173306332, 0.4503195185000447])
array('d', [0.8628965028315694, 0.8558229622195032, 0.09729015296133059, 0.4261538838233041, 0.18247495347014242, 0.7868251239508647, 0.7784639849469168, 0.49860463615919026, 0.8080630006641608, 0.7272807958140222])
array('d', [0.09729015296133059, 0.18247495347014242, 0.4261538838233041, 0.49860463615919026, 0.7272807958140222, 0.7784639849469168, 0.7868251239508647, 0.8080630006641608, 0.8558229622195032, 0.8628965028315694])


In [56]:
# example of deque
from collections import deque

dq = deque(range(10), maxlen=10)
print(dq)
dq.rotate(3)
print(dq)
dq.rotate(-4)
print(dq)
dq.appendleft(-1)
print(dq)
dq.extend([11, 22, 33])
print(dq)
dq.extendleft([10, 20, 30, 40])
print(dq)

deque([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([7, 8, 9, 0, 1, 2, 3, 4, 5, 6], maxlen=10)
deque([1, 2, 3, 4, 5, 6, 7, 8, 9, 0], maxlen=10)
deque([-1, 1, 2, 3, 4, 5, 6, 7, 8, 9], maxlen=10)
deque([3, 4, 5, 6, 7, 8, 9, 11, 22, 33], maxlen=10)
deque([40, 30, 20, 10, 3, 4, 5, 6, 7, 8], maxlen=10)


### 3. Dictionaries and Sets


##### Generic Mapping Types
- Mapping, MutableMapping: interfaces of dict
- Implementations of special mappings often extend dict or collections.UserDict
- keys must be hashable


##### Hashable
- an object is hashable:
    - if it has a hash value which never changes during lifetime; needs \_\_hash\_\_() method
    - can be compared to other objects; needs \_\_eq\_\_() method
- all immutable built-in objects are hashable
    - except tuple, tuple is hashable only if all its items are hashable
- user-defined types are hashable by default
    - hash value is id() and they all compare not equal
    - if implements custom \_\_eq()\_\_, the object is hashable only if all its attributes are immutable
    

##### Common Mapping Methods
- Handling Missing Keys with **setdefault**
    - e.g., index.setdefault(key, []).append(value)
- defaultdict: another elegant solution of handling missing keys
    - produce a defaultvalue whenever __getitem__ is passed a nonexisted key argument
    - default_factory: produce default values
    - only invoked for \_\_getitem\_\_ calls, e.g. dd.get(k) do not call the default_factory
- \_\_missing\_\_()
    - if provided \_\_missing\_\_ method, the dict.\_\_getitem\_\_ will call it when key is not found, instead of raising KeyError


##### Variations of dict
- collections.OrderedDict
    - maintains keys in insertion order; iter items in a predictable order
    - popitem pops the first item by default; popitem(last=True) pops the last item added
- collections.Counter
    - holds an integer count for each key
    - counter.update(), counter.most_common(n)
- collection.UserDict
    - pure python implementation, works like a standard dict
    - easier to create a new mapping type by extending UserDict rather than dict
    - UserDict does not inherit from dict, but has an internal dict instance (called data)
- Immutable Mappings -- MappingProxyType
    - builds a read-only mappingproxy instance from a dict
    
    
##### Set
- literal set syntax like {1, 2, 3} is both **faster** and more readable than calling the constructior set([1, 2, 3])
- set operations:
    - infix operators: &, |, -, ^, require both operands be sets 
    - methods: and, or, sub, xor, arguments can be iterables of any type
    - set predicates: in, <=, <, >=, >
    
    
##### dict and set under the Hood
- Hash tables in dictionaries
    - hash table is a sparse array, cells in a hash table are called "buckets"
    - python tries to keep at least 1/3 of the buckets empty; if too crowded, it is copied to to a larger place
- Hashes and equality
    - hash() works directly with built-in types; call \_\_hash\_\_() for user-defined types
    - if two objects compare equal, their hash values must be equal
- the hash table algorithm
    1. calls hash(search_key) to obtain the hash_value of search_key
    2. look up a bucket in the hash table by hash_value
        - 2.1. if bucket is empty, raise KeyError
        - 2.2. else get found_key, found_value in the found bucket
            - 2.2.1 if found_key == search_key, return found_value
            - 2.2.2 **hash collision**, lookup a different bucket (offset giving by the algorithm), repeat 2 
    - Note: the chance of collision is very low
- an object is hashable if all these requirements are met:
    1. supports the hash() function, and always returns the same value over the lifetime of the object
    2. supports the eq() function
    3. if a == b, then hash(a) == hash(b) must also be True
- Notes of dicts and hash tables
    - User-defined types are hashable by default, use id() as their hash value and they all compare not equal
    - dicts have significant memory overhead, -- hash tables must be **sparse** to work
    - key ordering depends on insertion order
    - adding items to a dict may change the order of existing keys
        - It's a bad idea that modifying the contents of dict while iterating through it

In [70]:
# exmaple of setdefault, defaultdict
import re
import collections

WORD_RE = re.compile('\w+')
file_path = '../../../../AI/NG-Coursera/mlng.md'

#index = {}
index = collections.defaultdict(list)
with open(file_path, encoding='utf-8') as fp:
    for line_no, line in enumerate(fp, 1):
        for match in WORD_RE.finditer(line):
            word = match.group()
            column_no = match.start() + 1
            location = (line_no, column_no)
            ''' 
            #method 1: get()
            occurences = index.get(word, [])
            occurences.append(location)
            index[word] = occurences
            '''
            ''' 
            #method 2: setdefault
            index.setdefault(word, []).append(location)
            '''
            #method 3: use defaultdict
            index[word].append(location)
#for word in sorted(index, key=str.upper):
#    print(word, index[word])

In [71]:
# example of dict.__missing__

class StrKeyDict0(dict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()
    
d = StrKeyDict0([('2', 'two'), ('4', 'four')])
print(d['2'])
print(d[4])
print(d.get(1, 'NA'))
#print(d[1])

two
four
NA


In [67]:
# example of UserDict
import collections

class StrKeyDict(collections.UserDict):
    def __missing__(self, key):
        if isinstance(key, str):
            raise KeyError(key)
        return self[str(key)]
    def __contains__(self, key):
        return str(key) in self.data
    def __setitem__(self, key, item):
        self.data[str(key)] = item
        
d = StrKeyDict([('2', 'two'), ('4', 'four')])
print(d['2'])
print(d[4])
print(d.get(1, 'NA'))
d[1] = 'one'
print(d[1])

two
four
NA
one


In [68]:
# example of Couter
import collections

ct = collections.Counter('abracadabra')
print(ct)
ct.update('aaaaazzz')
print(ct)
print(ct.most_common(2))

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
Counter({'a': 10, 'z': 3, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
[('a', 10), ('z', 3)]


### 4. Text versus Bytes


#####  Character
- character is **Unicode character**, python3 str, python2 unicode object
- the identity of a charactor => **code point**, is a number
- **encoding**: algorithm converts code points to byte sequences
    - e.g., code point for A (U+0041) is encoded as \X41 by utf-8 encoding
    - encode str to bytes for storage or transmission
- **decoding**: converting from bytes to code points
    - decode bytes to str to get human-readable text
    

##### Byte
- 2 built-in types:  **bytes** - immutable, **bytearray** - mutable, no literal syntax for bytearray
- each item is 0-255 integer
- 3 different displays
    - ASCII character, if in printable ASCII range
    - escape sequences, \t, \n, \r, \\ represents tab, newline, carriage return, \
    - hexadecimal escape sequence, for other bytes, \x00 -> null
- bytes and bytearray supports most str method (except formatting and others depends on unicode data)
- ways of creating bytes or bytearray:
    - bytes.fromhex()
    - call constructor, using different params
        - str and encoding 
        - iterable values from 0 to 255
        - single integer as size
        - object of buffer protocol
      
     
 - creating bytes or bytearray will always copy the bytes, but **memoryview** share memory
 - struct: parse packed bytes into a tuple of fileds, used with bytes, bytearray and memoryview objects

In [69]:
# example of encoding and decoding
b = b'caf\xc3\xa9' 
print(len(b))
s = b.decode('utf8')
print(s)
print(len(s))

5
café
4


In [11]:
# example of creating bytes
import array
b1 = bytes.fromhex('31 4B CE A9')
print(b1)
numbers = array.array('h', [-2, -1, 0, 1, 2, 97, 65])
b2 = bytes(numbers)
print(b2, len(b2), len(numbers))
numbers_1 = array.array('b', [-2, -1, 0, 1, 2, 97, 65])
b3 = bytes(numbers_1)
print(b3, len(b3), len(numbers_1))
b4 = bytes('abc', 'utf-8')
print(b4)
b5 = bytes(4)
print(b5)

b'1K\xce\xa9'
b'\xfe\xff\xff\xff\x00\x00\x01\x00\x02\x00a\x00A\x00' 14 7
b'\xfe\xff\x00\x01\x02aA' 7 7
b'abc'
b'\x00\x00\x00\x00'
